In [54]:
%%sh
pygmentize sklearn-boston-housing.py

import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import joblib
import argparse, os

if __name__ == '__main__':
        
    parser = argparse.ArgumentParser()
    parser.add_argument('--normalize', type=bool, default=False)
    parser.add_argument('--test-size', type=float, default=0.1)
    parser.add_argument('--random-state', type=int, default=123)
    parser.add_argument('--model-dir', type=str, default=os.environ['SM_MODEL_DIR'])
    parser.add_argument('--training', type=str, default=os.environ['SM_CHANNEL_TRAINING'])
    
    args, _ = parser.parse_known_args()
    normalize = args.normalize
    test_size = args.test_size
    random_state = args.random_state
    model_dir  = args.model_dir
    training_dir = args.training

    filename = os.path.join(training_dir, 'housing.csv')
    data = pd.read_csv(filename, delim_whitespace=True)
    labels = data

In [56]:
%%sh
pygmentize Dockerfile

FROM python:3.7

RUN pip3 install --no-cache scikit-learn numpy pandas joblib sagemaker-training 


In [57]:
%%sh
aws ecr create-repository --repository-name sklearn-custom


An error occurred (RepositoryAlreadyExistsException) when calling the CreateRepository operation: The repository with name 'sklearn-custom' already exists in the registry with id '613904931467'


In [ ]:
%%sh
export REGION=eu-west-1
export ACCOUNT_ID=`aws sts get-caller-identity --query Account --output text`
docker build -t sklearn-custom:sklearn -f Dockerfile .
export IMAGE_ID=`docker images -q sklearn-custom:sklearn`
docker tag $IMAGE_ID $ACCOUNT_ID.dkr.ecr.$REGION.amazonaws.com/sklearn-custom:sklearn
aws ecr get-login-password | docker login --username AWS --password-stdin $ACCOUNT_ID.dkr.ecr.$REGION.amazonaws.com/sklearn-custom:latest
docker push $ACCOUNT_ID.dkr.ecr.$REGION.amazonaws.com/sklearn-custom:sklearn

In [ ]:
import sagemaker
from sagemaker.sklearn import SKLearn

session = sagemaker.Session()
account_id = session.boto_session.client('sts').get_caller_identity()['Account']
region = session.boto_session.region_name

training = 'file://.'
output = 'file://.'

role = sagemaker.get_execution_role()
sk = SKLearn(entry_point='sklearn-boston-housing.py',
             image_name=account_id+'.dkr.ecr.'+region+'.amazonaws.com/sklearn-custom:sklearn',
             role=role,
             train_instance_count=1, 
             train_instance_type='local',
             output_path=output,
             hyperparameters={
                  'normalize': True,
                  'test-size': 0.1
              }
)

sk.fit({'training':training})

In [52]:
%%sh
tar tvfz model.tar.gz

-rw-r--r-- ec2-user/ec2-user 802 2020-05-19 08:16 model.joblib
